# 🚀 Hindi Voice Clone Turbo (Ultimate Edition)
यह नोटबुक **Coqui XTTS-v2** और **TTS 5** पर आधारित है। इसमें गूगल ड्राइव बैकअप और टर्बो लोड फीचर्स शामिल हैं।

### ✨ मुख्य फीचर्स:
- **Turbo Speed:** GPU का उपयोग करके बिजली की गति से ऑडियो जनरेट करना।
- **Google Drive Sync:** मॉडल को ड्राइव में बैकअप लें और वहीं से लोड करें।
- **Silence Remover:** फालतू सन्नाटा अपने आप हटा देता है।

In [ ]:
# @title 1. सेटअप और इंस्टॉलेशन (एक बार चलाएं)
!apt-get update -qq && apt-get install -y ffmpeg
!pip install TTS transformers<4.50.0 torch==2.1.0 torchaudio==2.1.0 pydub
print("✅ सेटअप पूरा हुआ! टर्बो मोड तैयार है।")

In [ ]:
# @title 📂 Backup & Load Model (Google Drive)
import os
import shutil
from google.colab import drive

if not os.path.exists("/content/drive"):
    drive.mount('/content/drive')

# --- UI FORM ---
Model_Name = "Hindi_Turbo_Model" # @param {type:"string"}
Action = "Load from Drive" # @param ["Backup to Drive", "Load from Drive"]

drive_path = f"/content/drive/MyDrive/{Model_Name}"
local_path = "/content/temp_model"

if Action == "Backup to Drive":
    if os.path.exists(local_path):
        print(f"🚀 {Model_Name} का बैकअप लिया जा रहा है...")
        shutil.copytree(local_path, drive_path, dirs_exist_ok=True)
        print(f"✅ बैकअप पूरा हुआ! यहाँ सेव है: {drive_path}")
    else:
        print("❌ एरर: कोलाब में कोई मॉडल नहीं मिला। पहले मॉडल डाउनलोड करें।")
else:
    if os.path.exists(drive_path):
        print(f"⚡ ड्राइव से {Model_Name} लोड हो रहा है...")
        shutil.copytree(drive_path, local_path, dirs_exist_ok=True)
        print("✅ मॉडल लोड हो गया! अब आप सीधे आवाज़ जनरेट कर सकते हैं।")
    else:
        print(f"❌ एरर: ड्राइव में '{Model_Name}' नहीं मिला।")

In [ ]:
# @title 2. आवाज़ जनरेट करें (Turbo Speed)
from TTS.api import TTS
from pydub import AudioSegment
from pydub.silence import split_on_silence
from google.colab import files

Text = "नमस्ते, आपकी आवाज़ सफलतापूर्वक क्लोन हो गई है।" # @param {type:"string"}
Voice_Sample = "sample.wav" # @param {type:"string"}

device = "cuda" if torch.cuda.is_available() else "cpu"

# अगर ड्राइव से लोड नहीं किया है, तो डिफ़ॉल्ट मॉडल डाउनलोड करें
if not os.path.exists("/content/temp_model/config.json"):
    print("🔄 डिफ़ॉल्ट मॉडल डाउनलोड हो रहा है...")
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
else:
    tts = TTS(model_path="/content/temp_model").to(device)

print("⚡ आवाज़ जनरेट हो रही है...")
tts.tts_to_file(text=Text, speaker_wav=Voice_Sample, language="hi", file_path="out.wav")

# साइलेंस रिमूवर लॉजिक
sound = AudioSegment.from_file("out.wav")
chunks = split_on_silence(sound, min_silence_len=400, silence_thresh=-45)
clean = AudioSegment.empty()
for c in chunks: clean += c
clean.export("final_output.wav", format="wav")

print("✅ सफलतापूर्वक पूर्ण!")
files.download("final_output.wav")

In [ ]:
# @title 📥 Optional: Download Specific Files (.pth / .json)
File_To_Download = "model.pth" # @param ["model.pth", "config.json", "vocab.json", "speakers_xtts.pth"]
path = f"/content/temp_model/{File_To_Download}"
if os.path.exists(path):
    files.download(path)
else:
    print("❌ फाइल नहीं मिली।")